## Этап обработки данных (ELT процесс)

Этап выполняется с целью подготовки данных для последующего обучения моделей (ноутбуки sklearn_models.ipynb, pytorch_models.ipynb) на данных,
извлекаемых из API https://i.centr.by/inforoads/api/v3/measurings.
Целью обучения является прогнозирование значения surface ['Closed' 'Dry' 'MoreWet' 'Wet' 'WetAndChemicals'] методом классификации.


In [ ]:
import requests
from io import StringIO
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
import numpy as np

### Извлечение данных (Extract)

In [2]:
# Загрузка данных с API
response = requests.get('https://i.centr.by/inforoads/api/v3/measurings')
print(response.status_code)

200


### Загрузка данных (Load)

In [3]:
# Преобразование в Pandas DataFrame
df = pd.DataFrame(response.json())

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   disId                           130 non-null    int64  
 1   date                            130 non-null    object 
 2   airTemperature                  129 non-null    float64
 3   humidity                        129 non-null    float64
 4   dewPoint                        129 non-null    float64
 5   isRain                          130 non-null    bool   
 6   windSpeed                       129 non-null    float64
 7   windDirection                   129 non-null    float64
 8   windCardinalDirection           130 non-null    object 
 9   windCardinalDirectionLocalized  130 non-null    object 
 10  precipitationAmount             112 non-null    float64
 11  precipitationIntensity          128 non-null    float64
 12  snowHeight                      108 

### Трансформация данных (Transform)

#### Очистка данных

In [5]:
# Удалим дублирующие колонки
df_d = df.drop(
    ['windCardinalDirectionLocalized', 'precipitationLocalized', 'warningLocalized', 'surfaceLocalized'],
    axis='columns'                
)

# Удалим колонки с отсутствующими значениями, которые могут значительно сократить выборку
df_c = df_d.drop(
    ['pressure', 'temperatureTrend', 'frequencyOfBlackIce', 'warning'],
    axis='columns'
)

# Удалим строки с отсутствующими значениями
df_r = df_c.dropna()

#### Преобразование данных

In [6]:
# Выберем колонки с категориальными значениями для one-hot кодирования
categirical = df_r[['isRain', 'windCardinalDirection', 'precipitation']]

# Выберем колонку с метками
surface = df_r['surface']
surface.info()

# Удалим колонки с категориальными значениями и данными для меток из исходного набора
df_r = df_r.drop(['disId', 'date', 'isRain', 'windCardinalDirection', 'precipitation', 'surface'], axis='columns')

# Кодирование категориальных переменных (one-hot encoding)
oh_enc = OneHotEncoder()
oh_enc.fit(categirical)
encoded = oh_enc.transform(categirical).toarray()
print(oh_enc.categories_)

<class 'pandas.core.series.Series'>
Index: 104 entries, 1 to 127
Series name: surface
Non-Null Count  Dtype 
--------------  ----- 
104 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB
[array([False,  True]), array(['E', 'N', 'NE', 'NW', 'S', 'SE', 'SW', 'W'], dtype=object), array(['Clear', 'Cloudy', 'NoPrecipitation', 'WeakRain'], dtype=object)]


In [7]:
# Преобразуем закодированные колонки в DataFrame
encoded_df = pd.DataFrame(encoded)
encoded_df.head()
df_concat = pd.concat([df_r.reset_index(drop=True), encoded_df], axis=1)

In [8]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   airTemperature          104 non-null    float64
 1   humidity                104 non-null    float64
 2   dewPoint                104 non-null    float64
 3   windSpeed               104 non-null    float64
 4   windDirection           104 non-null    float64
 5   precipitationAmount     104 non-null    float64
 6   precipitationIntensity  104 non-null    float64
 7   snowHeight              104 non-null    float64
 8   visibility              104 non-null    float64
 9   roadTemperature         104 non-null    float64
 10  soilTemperature         104 non-null    float64
 11  freezingPointLiquidus   104 non-null    float64
 12  freezingPointSolidus    104 non-null    float64
 13  chemicalConcentration   104 non-null    float64
 14  chemicalQuantity        104 non-null    fl

#### Разделение данных

In [ ]:
# Закодируем метки - прогнозируемое категориальное значение (one-hot encoding)
lb = LabelBinarizer()
oh_labels = lb.fit_transform(surface)
ordinal_labels = np.argmax(oh_labels, axis=1)[:, np.newaxis]
labels = np.hstack((ordinal_labels, oh_labels))


In [14]:
print(lb.classes_)

['Closed' 'Dry' 'MoreWet' 'Wet' 'WetAndChemicals']


In [10]:
# Сохраним метки в csv-файл
df_labels = pd.DataFrame(labels)
df_labels.to_csv('labels.csv', index=False)

In [11]:
# Сохраним набор данных в csv-файл
df_concat.to_csv('dataset.csv', index=False)